# Merge land cover for publication figure

This notebook merges the high-resolution LISA land cover with the lower-resolution CORINE land cover to create a land cover raster that covers a large extent to depict the surrounding landscape around the study area in a publication figure. <br />

The 1.07 version of [basemap](https://github.com/matplotlib/basemap) is not compatible with the 0.36 version of [rasterio](https://github.com/mapbox/rasterio), which is required to merge the rasters. Therefore, this notebook must be run in a different environment than the other notebooks.

#### Import statements

In [4]:
from geo.models import Raster
import rasterio
from rasterio.merge import merge
import os.path

#### Load input raster objects

In [6]:
landcover_study_area = Raster.objects.get(name='landcover study area')
landcover_background = Raster.objects.get(name='landcover corine mapped lisa')

#### Merge land cover

s1 = rasterio.open(landcover_study_area.filepath, mode='r')
s2 = rasterio.open(landcover_background.filepath, mode='r')
landcover, transform = merge(sources=[s1, s2], res=(5., 5.), nodata=255)

#### Write merged raster to disk

In [10]:
output_filepath = os.path.join(os.path.dirname(landcover_background.filepath), "landcover_merged_publication.tif")
with rasterio.open(path=output_filepath, mode='w', driver='GTiff', 
                   width=landcover.shape[2], height=landcover.shape[1], count=landcover.shape[0], 
                   crs="+init=epsg:31254", transform=transform, dtype=landcover.dtype, nodata=int(s1.nodata)) as dest:
    dest.write(landcover[0], 1)

#### Close input raster files

In [11]:
s1.close()
s2.close()

#### Save new raster object to database

In [13]:
Raster(name='landcover merged publication', filepath=output_filepath).save()